In [1]:
# Block 1: Importing Libraries
import tarfile
import pandas as pd
import librosa
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import pickle
from sklearn.preprocessing import StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is being used.")
else:
    print("No GPU found. Running on CPU.")
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")


No GPU found. Running on CPU.
GPU: NVIDIA GeForce RTX 4080 Laptop GPU is available.


[]

In [2]:
import shutil

def extract_tar(tar_file, target_dir):
    if os.path.exists(target_dir):
        user_input = input(f"The directory '{target_dir}' already exists. Do you want to skip extraction? (y/n): ")
        if user_input.lower() == 'y':
            print(f"Skipping extraction of {tar_file}.")
            return
        else:
            print(f"Overwriting the existing directory '{target_dir}'.")
            shutil.rmtree(target_dir)
    
    with tarfile.open(tar_file, 'r') as tar:
        tar.extractall(target_dir)
    
    # Remove residue "._" hidden files
    for root, dirs, files in os.walk(target_dir):
        for file in files:
            if file.startswith("._"):
                os.remove(os.path.join(root, file))

extract_tar('train_mp3s.tar', 'train_mp3s')
extract_tar('test_mp3s.tar', 'test_mp3s')

train_labels = np.loadtxt('train_label.txt', dtype=int)

Skipping extraction of train_mp3s.tar.
Skipping extraction of test_mp3s.tar.


In [3]:
# Block 3: Preprocessing Functions
def preprocess_audio(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        print(f"Loaded audio file: {file_path}")
        
        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
        
        # Extract mel-spectrogram features
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spec_scaled = np.mean(mel_spec_db.T, axis=0)
        
        # Extract chroma features
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        chroma_scaled = np.mean(chroma.T, axis=0)
        
        # Extract spectral contrast features
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        contrast_scaled = np.mean(contrast.T, axis=0)
        
        # Extract tonnetz features
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sample_rate)
        tonnetz_scaled = np.mean(tonnetz.T, axis=0)
        return mel_spec_scaled
        # Concatenate all features
        features = np.concatenate((mfccs_scaled, mel_spec_scaled, chroma_scaled, contrast_scaled, tonnetz_scaled))
        print(f"Extracted features: {features.shape}")
        return features
        return mel_spec_scaled
    except Exception as e:
        print(f"Error processing file: {file_path}")
        print(f"Error message: {str(e)}")
        return None

def prepare_data(directory, feature_file):
    if os.path.exists(feature_file):
        print(f"Loading features from {feature_file}")
        features = np.load(feature_file)
    else:
        file_paths = [os.path.join(directory, f"{i}.mp3") for i in range(len(os.listdir(directory)))]

        with ThreadPoolExecutor() as executor:
            features = list(executor.map(preprocess_audio, file_paths))

        features = [feature for feature in features if feature is not None]

        if len(features) == 0:
            print("No valid features extracted. Please check the data.")
            return None

        features = np.array(features)
        np.save(feature_file, features)
        print(f"Features saved to {feature_file}")

    print(f"Processed {len(features)} audio files")
    return features

In [4]:

# Block 4: Preparing Data

train_feature_file = 'train_features.npy'
train_features = prepare_data('train_mp3s/train_mp3s', train_feature_file)

if train_features is not None:
    print(f"Train features shape: {train_features.shape}")
else:
    print("No valid train features. Please check the data.")

test_feature_file = 'test_features.npy'
test_features = prepare_data('test_mp3s/test_mp3s', test_feature_file)

if test_features is not None:
    print(f"Test features shape: {test_features.shape}")
else:
    print("No valid test features. Please check the data.")

if train_features is not None and test_features is not None:
    train_labels = np.array([int(label) for label in train_labels])
    print(f"Train labels shape: {train_labels.shape}")

    print(f"Number of training features: {len(train_features)}")
    print(f"Number of training labels: {len(train_labels)}")
    print(f"Number of test features: {len(test_features)}")

    # Normalize the features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    test_features = scaler.transform(test_features)

    # Save the scaler for future use
    scaler_filename = 'scaler.pkl'
    with open(scaler_filename, 'wb') as file:
        pickle.dump(scaler, file)
    print(f"Scaler saved to {scaler_filename}")

    # Reshape the features for LSTM input
    train_features = train_features.reshape((train_features.shape[0], train_features.shape[1], 1))
    test_features = test_features.reshape((test_features.shape[0], test_features.shape[1], 1))
else:
    print("Insufficient data. Please check the input files.")



In [ ]:
# Block 5: Model Training and Prediction
if len(train_features) > 0:
    train_data, val_data, train_labels, val_labels = train_test_split(train_features, train_labels, test_size=0.3, random_state=42)

    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=(train_features.shape[1], 1)),
        MaxPooling1D(2),
        LSTM(128),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(train_data, train_labels, validation_data=(val_data, val_labels),
              epochs=30, batch_size=32, callbacks=[early_stopping])

    predictions = model.predict(test_features)
    predicted_labels = np.argmax(predictions, axis=1)

    submission = pd.DataFrame({'id': range(len(predicted_labels)), 'category': predicted_labels})
    submission.to_csv('submission.csv', index=False)

else:
    print("No training features available. Please check the data.")

TypeError: object of type 'NoneType' has no len()